In [1]:

####################################################################################################
####################################################################################################

import datetime
import sys
import glob
import json
import os.path
import pprint

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd

from IPython.display import display
#from IPython.display import Image
from PIL import Image

#
import keras
from keras import backend as K
from keras import layers
from keras import regularizers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

sys.path.append(os.path.abspath("../../"))
sys.path.append(os.path.abspath("../../pax_utils"))
from s1s2_utils import *
from keras_utils.config_utils import *

from model_xy_s2waveforms_dnn import *



Using TensorFlow backend.


In [2]:

####################################################################################################
####################################################################################################
    
nEventsFile     = 25000
nEventsTrain    = 20000
#nEventsTrain    = 1

iEventStart     = 0
iEventEnd       = iEventStart + nEventsFile

s2_window_max   = 2300
resample_factor = 46
resample_factor = 230

n_timesteps     = int(s2_window_max / resample_factor)

assert(s2_window_max % resample_factor == 0)


####################################################################################################
####################################################################################################

#file_out_input = 'train/array_train_input_events%06d-%06d_timesteps%04d' % (iEventStart, iEventEnd - 1, n_timesteps) + '.npy'
#file_out_truth = 'train/array_train_truth_events%06d-%06d_timesteps%04d' % (iEventStart, iEventEnd - 1, n_timesteps) + '.npy'

file_out_input = 'train/array_train_input_events000000-199999_timesteps0010.npy'
file_out_truth = 'train/array_train_truth_events000000-199999_timesteps0010.npy'

#file_out_input = 'array_train_input_events000000-199999_timesteps0020.npy'
#file_out_truth = 'array_train_truth_events000000-199999_timesteps0020.npy'

#file_out_input = 'array_train_input_events000000-199999_timesteps0025.npy'
#file_out_truth = 'array_train_truth_events000000-199999_timesteps0025.npy'

#file_out_input = 'data_all.npy'
#file_out_truth = 'truth_all.npy'

nEventsTrain   = 100000


####################################################################################################
####################################################################################################

train_data  = np.load(file_out_input)
train_truth = np.load(file_out_truth)

train_data  = train_data [0:nEventsTrain, :]
train_truth = train_truth[0:nEventsTrain, :]

print(train_data.shape)
print(train_truth.shape)



FileNotFoundError: [Errno 2] No such file or directory: 'array_train_input_events000000-199999_timesteps0010.npy'

In [3]:

####################################################################################################
####################################################################################################

n_events     = nEventsTrain
n_channels   = 127
n_outputs    = 2

#n_timesteps  = 1

go_backwards = False
unroll       = False


####################################################################################################
# Training Data
####################################################################################################

print()
print("Channels:         " + str(n_channels))
print("Timesteps:        " + str(n_timesteps))
print("Outputs:          " + str(n_outputs) )
print("Input data shape: " + str(train_data.shape ))
print("Truth data shape: " + str(train_truth.shape))
print()




Channels:         127
Timesteps:        10
Outputs:          2
Input data shape: (100000, 1270)
Truth data shape: (100000, 2)



In [4]:

####################################################################################################
####################################################################################################

model, model_name = dnnModel(n_channels, n_timesteps, n_outputs, 'elu', 0.00005)


######################################################################################
######################################################################################


print()
print("Model Summary:")
model.summary()
print()





Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1270)              1614170   
_________________________________________________________________
dense_2 (Dense)              (None, 1270)              1614170   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1270)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 127)               161417    
_________________________________________________________________
dropout_2 (Dropout)          (None, 127)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 256       
Total params: 3,390,013
Trainable params: 3,390,013
Non-trainable params: 0
__________________________________________________

In [5]:

######################################################################################
######################################################################################

# to do: reset model 

epochs = 10
#epochs = 1

history = model.fit(
    train_data,
    train_truth,
    batch_size=64,
    epochs=epochs,
    verbose=True
)


######################################################################################
######################################################################################

dct_history = history.history

loss = dct_history['loss'][epochs-1]
loss = int(round(loss*100, 0))
loss = 'loss' + str(loss)

acc  = dct_history['acc'][epochs-1]
acc  = int(round(acc*1e4, 0))
acc  = 'acc%04d' % acc


print("Training Complete.")


Epoch 1/10
100000/100000 [==============================] - 371s 4ms/step - loss: 3.1651 - acc: 0.9869
Epoch 2/10
100000/100000 [==============================] - 442s 4ms/step - loss: 1.1438 - acc: 0.9899
Epoch 3/10
100000/100000 [==============================] - 415s 4ms/step - loss: 1.1146 - acc: 0.9896
Epoch 4/10
100000/100000 [==============================] - 384s 4ms/step - loss: 1.0475 - acc: 0.9898
Epoch 5/10
100000/100000 [==============================] - 355s 4ms/step - loss: 1.0220 - acc: 0.9901
Epoch 6/10
100000/100000 [==============================] - 336s 3ms/step - loss: 0.9527 - acc: 0.9903
Epoch 7/10
100000/100000 [==============================] - 355s 4ms/step - loss: 0.9062 - acc: 0.9906
Epoch 8/10
100000/100000 [==============================] - 337s 3ms/step - loss: 0.8333 - acc: 0.9908
Epoch 9/10
100000/100000 [==============================] - 349s 3ms/step - loss: 0.7987 - acc: 0.9908
Epoch 10/10
100000/100000 [==============================] - 375s 4ms/ste

In [7]:

######################################################################################
######################################################################################

config     = model.get_config()
str_layers = layer_desc = getModelDescription(config)   


######################################################################################
# Save Model
######################################################################################

folder   = "models/"    
desc     = model_name + '_' + acc + '_epochs' + str(epochs) + '_' + layer_desc 
name_h5  = folder + desc + '.h5'
name_png = name_h5.replace('.h5', '.png')
name_cfg = name_h5.replace('.h5', '.json')


######################################################################################
# Save
######################################################################################

model.save(name_h5, overwrite=True)

plot_model(model, to_file=name_png, show_layer_names=True, show_shapes=True)

with open(name_cfg, 'w') as fp: json.dump(config, fp)
with open(name_cfg, 'r') as fp: config_test = json.load(fp)
    

print("\n" + name_h5 + "\n")

#print()
#display(Image.open(name_png))
#print()




models/dnn_xy_s2waveforms_elu_acc9915_epochs10_layers-1270-1270-127-2.h5

